<a href="https://colab.research.google.com/github/fdac25/trading/blob/main/src/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# WIP - still converting to fit our application
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

In [ ]:

df = pd.read_csv('input csv here')
df.head()

In [ ]:
# To remove patterns from the text
def clean_text(d):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', d)
    return text

In [ ]:
# this is where we want to put the names of the stocks probably, depending on article
names = ['','']

In [ ]:
# to remove stop words from the text
def clean_stopword(d):
    stop_words = stopwords.words('english')
    for name in names:
        stop_words.append(name)
    return " ".join([w.lower() for w in d.split() if w.lower() not in stop_words and len(w) > 1])

In [ ]:
# To tokenize the text
def tokenize(d):
    return word_tokenize(d)

In [ ]:
# Applying all the functions
df['final_text']= df.text.apply(clean_text).apply(clean_stopword).apply(tokenize)
df.final_text.head()

In [ ]:
# Converting tokens to strings
for i in range(len(df)):
    df['final_text'][i] = " ".join(df['final_text'][i])

In [ ]:
# Adding neagtivereason to strings
df['final_text_2'] = df['negativereason'].fillna('') + ' ' + df['final_text']
df[['final_text' , 'final_text_2']]

In [ ]:
# Picking out X and y from the data
X = df.final_text_2
y = df.sentiment

In [ ]:
# Splliting the X and y into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2, random_state = 4)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
# TF-IDF Vectorizer
def tfidf(words):
    tfidf_vectorizer = TfidfVectorizer()
    data_feature = tfidf_vectorizer.fit_transform(words)
    return data_feature, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train.tolist())
X_test_tfidf = tfidf_vectorizer.transform(X_test.tolist())

In [ ]:
# Logistic Regression Model to predict
lr_tfidf = LogisticRegression(random_state=42,solver = 'liblinear')
lr_tfidf.fit(X_train_tfidf, y_train)
y_predicted_lr = lr_tfidf.predict(X_test_tfidf)

In [ ]:
# Function to print accuaracy, precision and recall
def score_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted,average= 'macro')
    recall = recall_score(y_test, y_predicted,average='macro')
    f1 = f1_score(y_test, y_predicted,average='macro')
    print("accuracy = %0.3f, precision = %0.3f, recall = %0.3f, f1 = %0.3f" % (accuracy, precision, recall, f1))

In [ ]:
score_metrics(y_test, y_predicted_lr)

In [ ]:
# Function to plot Confusion Matrix
def plot_confusion_matrix(y_test, y_predicted, title='Confusion Matrix'):
    cm = confusion_matrix(y_test, y_predicted)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm,annot=True, fmt='.20g')
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(y_test, y_predicted_lr)

In [ ]:
t = input("Enter String : ")
t={'text':t}
t = pd.DataFrame(t,index=[0])
t = tfidf_vectorizer.transform(t.text.tolist())
out = lr_tfidf.predict(t)
print('Sentiment Value : ',str(out))